In [1]:
pip install --upgrade pillow

In [2]:
import numpy as np 
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import random 

from PIL import Image
from itertools import groupby

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

import tensorflow as tf 
from tensorflow import keras
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.callbacks import EarlyStopping



#**CNN을 활용한 손글씨 인식 계산기**

**주제 선정이유** 

섹션4에서 배운 딥러닝 기법중 이미지 처리 부분이 제일 흥미로웠습니다.  배운 부분을 활용해서 사용자 편의나 교육에 활용할 어플을 만들수 없을까 생각하다가 손으로 수학문제를 풀고 인식시켜 채점을 해주는 어플을 생각했습니다. 


이번 프로젝트를 진행하면서 수식을 입력하면 정답을 알려주는데까지 구현했습니다. 데이터셋 구하기가 너무 힘들어서 아직 사칙연산 밖에 안되지만 이를 바탕으로 확장해나가고 페이지도 만들어서 어플로 발전시킬 수 있지 않을까 싶습니다. 


# 데이터 셋 설명

> 0~9까지의 아라비아 숫자 및 사칙연산자(+, -, /, *) 총 85,709개의 이미지 

> 각 이미지 크기는 28x28 픽셀 

> 연산자 레이블 :

*  10 = /
*  11 = +
*  12 = -
*  13 = *




In [3]:
 # 시드 고정
np.random.seed(2) 
random.seed(2)
tf.random.set_seed(2)
initializer = tf.keras.initializers.GlorotUniform(seed=2)


In [4]:
### 데이터 전처리 및 학습, 훈련 나누기 
def load_data(): 

    dataset = pd.read_csv("/content/drive/MyDrive/section4_data/dataset.csv")

    y = dataset["label"] 
    X = dataset.drop(["label"], axis = 1)

    # 이미지 스케일링 
    X = X / 255.0

    # 4차원 텐서에 맞춰 리쉐입 [미니배치사이즈, 높이, 폭, 채널]
    X = X.values.reshape(-1,28,28,1)

    # label 원핫 인코딩 클래스 개수 14개 
    label = to_categorical(y, num_classes = 14)

    # 데이터 나눠주기 
    X_train, X_val, y_train, y_val = train_test_split(X, label, test_size = 0.15 , random_state = 2, stratify = y)

    # print(f'학습 데이터 갯수 : {len(X_train)}, 검증 데이터 갯수 : {len(X_val)}\n\n') 
    # sns.countplot(dataset["label"]) # 클래스별 이미지 갯수 
    return X_train, X_val, y_train, y_val


# CNN 모델 구축, 평가 및 저장 

> 모델 구축 

* hidden layer : 2개의 컨볼루션층, 풀링층, 드랍아웃층으로 구성 

* fully connected layer : 이미지 입력을 위한 flatten, 256의 덴스층, 드랍아웃층, 클래스 예측을 위해 softmax를 활성화함수로 하는 마지막 덴스층 
---

> 과적합 방지와 일반화를 위해 데이터 증강과 kFold로 5번의 cross validation 수행 




> 마지막으로 훈련한 최종모델 저장 


In [5]:

### 모델 설정 

def define_model(): 
    model = Sequential()

    # 레이어 1 
    model.add(Conv2D(filters = 32, kernel_size = (5,5), padding = "Same", activation = "relu", input_shape = (28, 28, 1)))
    model.add(Conv2D(filters = 32, kernel_size = (5,5), padding = "Same", activation = "relu"))
    model.add(MaxPool2D(pool_size = (2,2)))
    model.add(Dropout(0.25))

    # 레이어 2 
    model.add(Conv2D(filters = 64, kernel_size = (3,3), padding = "Same", activation = "relu"))
    model.add(Conv2D(filters = 64, kernel_size = (3,3), padding = "Same", activation = "relu"))
    model.add(MaxPool2D(pool_size = (2,2)))
    model.add(Dropout(0.25))


    #fully connected 레이어 
    model.add(Flatten()) # flatten 으로 하나의 배열로 펴줌 
    model.add(Dense(256, activation = "relu"))
    model.add(Dropout(0.25))
    model.add(Dense(14, activation = "softmax", kernel_initializer=initializer)) # 아웃풋 클래스 14개

    # 컴파일 
    model.compile(optimizer = 'adam', loss = "categorical_crossentropy", metrics = ["accuracy"])

    return model 



In [6]:
model = define_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 32)        832       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 32)        25632     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 64)          0

In [7]:
### 데이터 제너레이터 정의 

def data_generator():
    # 데이터 augmentation 
    datagen = ImageDataGenerator(
            rotation_range=10,  # 이미지 회전 각도 
            zoom_range = 0.1, # zoom 범위 지정 
            width_shift_range=0.1,  # 수평이동 범위 
            height_shift_range=0.1,  # 수직이동 범위 
            horizontal_flip=True,  # 수평 flip
            vertical_flip=True)  # 수직 flip

    return datagen

In [8]:
### kFold를 적용한 모델 학습 및 평가 
def evaluate_model(X_data, y_data, n_folds=5): 

    scores, histories = list(), list() # score, history 리스트 만들기 

    kfold = KFold(n_folds, shuffle=True, random_state=2) # CV를 위한 kFold 정의 

    datagen = data_generator() # 데이터 제너레이터 불러오기 
    datagen.fit(X_data) # X 데이터에 fit 

    for train_ix, test_ix in kfold.split(X_data, y_data):

      model = define_model() # 모델 불러오기 

      # early stopping 지정  
      early_stop = EarlyStopping(monitor = "loss", 
                                patience = 3, # 학습진행에도 값의 개선이 없을경우 몇번 epoch하고 멈출지 설정
                                verbose = 1)
                                      
      history = model.fit_generator(datagen.flow(X_data[train_ix], y_data[train_ix], batch_size=64),
                                    epochs = 20,
                                    verbose = 0,
                                    callbacks=[early_stop])

      # evaluate
      _, acc = model.evaluate(X_data[test_ix], y_data[test_ix], verbose=0)

      # 스코어, 히스토리 저장 
      scores.append(round(acc, 3))
      histories.append(history)

    return scores, histories


In [34]:
### test셋에 적용 후 final 모델 저장 

def test_run_save():

    # 데이터 불러오기 
    X_train, X_test, y_train, y_test = load_data()

    # kFold로 evaluate 수행 
    scores, histories = evaluate_model(X_train, y_train)
    print('kFold score : ', scores)
    print('kFold average score : ', np.mean(scores))

    # 데이터 제너레이터 불러오기  
    datagen = data_generator() 
    datagen.fit(X_train)
    datagen.fit(X_test)

    # 모델 불러오기 
    model = define_model()
    # fit 
    history = model.fit_generator(datagen.flow(X_train, y_train, batch_size=64), epochs=15, verbose=0)
    # test셋으로 evaluate
    loss, acc = model.evaluate(X_test, y_test, verbose=0)
    print('\ntest loss : {:0.3f}, test Acc : {:0.3f}'.format(loss, acc))

    # 모델 저장 
    model.save('final_model.h5')

In [35]:
test_run_save() # 실행 

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


kFold score :  [0.981, 0.979, 0.973, 0.979, 0.971]
kFold average score :  0.9766

test loss : 0.066, test Acc : 0.978


# 모델 예측 

> 이미지 안의 요소 분리 

* 계산을 위해 예측할 이미지를 각각 숫자와 연산자로 분리하는 과정 필요 

* 값이 있는 열을 찾기 위해 가로열에서 0이 아닌 값이 있는 연속열들을 찾아서 분리 

* 분리한 요소들을 전처리 후 모델 인풋에 맞춰 리사이즈 

* 저장한 모델 불러와서 클래스 예측 수행 


In [11]:
### 새롭게 예측할 이미지 전처리 

def predict_image(image):
    # 모델에 넣기위해 28x28픽셀로 리사이즈 
    w = image.size[0]
    h = image.size[1]
    r = w / h # 가로세로 비율 
    new_w = int(r * 28)
    new_h = 28
    new_image = image.resize((new_w, new_h))

    # 넘파이 배열로 변경
    new_image_arr = np.array(new_image)

    # 배경을 0(검정색)으로 만들기
    new_inv_image_arr = 255 - new_image_arr

    # 이미지 스케일링 
    final_image_arr = new_inv_image_arr / 255.0

    # 이미지를 각각 숫자로 분할 
    # 이미지 배열에서 0이 아닌 연속 열을 찾고 그룹화해서 각 배열로 분류 
    m = final_image_arr.any(0)
    out = [final_image_arr[:,[*g]] for k, g in groupby(np.arange(len(m)), lambda x: m[x] != 0) if k]

    num_of_elements = len(out)
    elements_list = []

    # 분류한 배열을 인풋차원과 일치하도록 크기 조정
    for x in range(0, num_of_elements):

        img = out[x] # 각 요소 배열별 반복 
        
        # 분류 할때 없어진 가로의 0값 열 다시 채워주기

        width = img.shape[1] # 요소 배열의 폭 
        filler = (final_image_arr.shape[0] - width) / 2 # 양쪽에 붙여줘야 하니까 나누기 2
        
        if filler.is_integer() == False:    # filler가 홀수면 오른쪽에 + 1 
            filler_l = int(filler)
            filler_r = int(filler) + 1
        else:                               
            filler_l = int(filler)
            filler_r = int(filler)

        # 원래이미지 height, 계산한 filler로 width로 하는 zero배열 생성 
        arr_l = np.zeros((final_image_arr.shape[0], filler_l)) # left
        arr_r = np.zeros((final_image_arr.shape[0], filler_r)) # right
        
        # filler와 이미지 합쳐서 원래 이미지 크기로 
        help_ = np.concatenate((arr_l, img), axis= 1)
        element_arr = np.concatenate((help_, arr_r), axis= 1)
        
        # 크기 복원한 배열 리사이즈 (높이, 폭, 채널)
        element_arr.resize(28, 28, 1)

        # 리사이즈 끝난 각 배열들 리스트로 넣기 
        elements_list.append(element_arr)

    # 넘파이 배열로 변경 
    elements_array = np.array(elements_list)

    # 모델 인풋에 맞도록 4차원 텐서로 리쉐입 
    elements_array = elements_array.reshape(-1, 28, 28, 1)

    # 저장한 모델불러와서 predict
    model = keras.models.load_model("/content/final_model.h5")
    elements_pred = model.predict(elements_array) # softmax로 구한 각 클래스별 확률 
    class_pred = np.argmax(elements_pred, axis = 1) # 확률이 가장 높은 클래스 선택 

    return class_pred

# 계산기 만들기

>예측한 클래스 값을 이용해서 계산하는 부분

---


수식을 만들기 위해 숫자와 연산자로 변환

숫자는 자릿수에 맞춰서 변환 

example:
```
[1,0]
10 

[10]
/   (나누기 기호로)
```


마지막으로 숫자와 연산자를 합쳐 수식 문자열로 만들고 eval함수로 계산


In [12]:
### 계산기 부분 만들기 

# 예측한 클래스 수식으로 변경 
def math_expression(pred_arr):
    
    op = {
              10,   # = "/"
              11,   # = "+"
              12,   # = "-"
              13    # = "*"
                  }   
    
    m_exp = []
    temp = []
        
    # 리턴받은 클래스 리스트에서 각각 숫자와 기호를 구분 
    for item in pred_arr:
        if item not in op: # 연산자가 아니면 
            temp.append(item) # 숫자만 들어가는 리스트 
        else:
            m_exp.append(temp) # 다음 수식이 나오기 전까지의 숫자 리스트 
            m_exp.append(item) # 기호
            temp = []
    if temp:
        m_exp.append(temp)
        
    # 구분한 클래스를 숫자와 연산자로 변환 
    i = 0
    num = 0
    for item in m_exp: # 구분한 리스트 돌면서 
        if type(item) == list: # 타입이 리스트 일경우 (즉, 두자리 수 이상인 경우)
            if not item:
                m_exp[i] = ""
                i = i + 1
            else:
                num_len = len(item) # 숫자의 자릿수 길이 
                for digit in item: # 두자리수 이상인 리스트 안을 돌면서 
                    num_len = num_len - 1  # 자릿수 파악을 위해 자릿수에서 1을 빼서 10의 지수로 둠 (10의 0승=1, 10의 1승=10...)
                    num = num + ((10 ** num_len) * digit) # 자릿수 파악을 위한 수와 현재 숫자를 곱하고 더해줌 > 자릿수 별로 수를 구하고 합침
                m_exp[i] = str(num) # string 형태로 저장 
                num = 0
                i = i + 1 
        else:  # 한자리 숫자거나 연산자 경우 
            m_exp[i] = str(item)
            # 클래스에 맞춘 연산자로 대체 
            m_exp[i] = m_exp[i].replace("10","/")
            m_exp[i] = m_exp[i].replace("11","+")
            m_exp[i] = m_exp[i].replace("12","-")
            m_exp[i] = m_exp[i].replace("13","*")
            
            i = i + 1
    
    
    # 마지막으로 계산을 위해서 숫자와 연산자로 변환한 리스트를 하나의 문자열로 
    separator = ' '
    m_exp_str = separator.join(m_exp)
    
    return (m_exp_str)



In [13]:

# 변경한 수학식 계산 
def eval_math(exp_str):
    while True: # 계산가능 할때 
        try:
            answer = eval(exp_str)   # 입력받은 수식 정답 계산 
            answer = round(answer, 2) 
            equation  = exp_str + " = " + str(answer) # 등식으로 표현 
            print(equation)  
            break

        except SyntaxError: # 계산 불가인 경우 (ex 연산자가 맨 처음 있거나 맨 마지막에 있는 등의 불가)
            print("잘못된 수식이 입력되었습니다")
            print("수식을 확인해주세요")
            print(exp_str)
            break


In [55]:
# 예측할 이미지 불러오기 
image = Image.open("/content/drive/MyDrive/section4_data/img5.jpg").convert("L") # gray scale로 

pred_image = predict_image(image)# 예측한 클래스값 리턴 
print(pred_image) 

[ 6  2 10  8 13  5 11  7 12  3]


In [56]:
# 예측한 클래스 값 수학식으로 변경 
m_exp_str = math_expression(pred_image)

eval_math(m_exp_str)

62 / 8 * 5 + 7 - 3 = 42.75


**보완할점**

더 다양한 계산식 할 수 있게 만들기 (데이터를 어디서 구할지.. ㅜㅜ 많이 찾아봐야 할듯)

비슷하게 생긴 숫자 예측 실패 > 실패시 맞는 레이블링으로 모델 업데이트를 하는 방향으로 학습 



In [16]:
pip freeze > requirements.txt
